In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(44) # better for debugging to have seed 
import time
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tiny-arabic-poetry/arabic_poems.txt
/kaggle/input/arabic-poetry/arabic_poems.txt


In [2]:
with open('../input/tiny-arabic-poetry/arabic_poems.txt', 'r', encoding='utf-8') as f:
    poems = f.read()

In [3]:
print("Number of characters in dataset = ", len(poems))

Number of characters in dataset =  10000009


In [4]:
poems[:1000]

' مولاي قل لي اين ما قد كان من عهد وثيق حاشاك ان تنسي الذي بيني وبينك من حقوق ما مثل وجهك ذا الجمي ل يكون من اهل العقوق يبدو فيشرق لعيو ن ضحي ويشرقني بريقي وزعمت انك زاءري فتركت عيني لطريق وجعلتني ابكي علي ك من الغروب الي الشروق لو ان لي عينا تنا م قنعت بالطيف الطروق سقيا لايام الوصا ل وذلك العيش الانيق\nرَحِمَ اللَهُ جَعفَراً فَلَقَد كا\nنَ أَبِيّاً شَهماً وَكانَ رَحيما\nمُثِّلَ المَوتُ بَينَ عَينَيهِ وَالذُلُّ\nفَكُلّاً رَآهُ خَطباً عَظيما\nثُمَّ سارَت بِهِ الحَمِيَّةُ قُدماً\nفَأَماتَ العِدى وَماتَ كَريما\nشَهِدتُ بِأَنّي عَبدُ مَغناكُمُ الَّذي\nعَلى بابِكُم أَرضى حِجابَكُمُ عَنّي\nفَإِن شَنَّعَ الأَعداءُ عَنّي بِضِدِّهِ\nفَلا تَشهَدوا إِلّا بِمَسموعِكُم مِنّي\n\n قيل لي لم ذمت كل البرايا وهجوت الانام هجوا قبيحا قلت هب اني كذبت عليهم فاروني من يستحق المديحا\nلعمري يا عميد وأنت أدرى\nبحال الشعب فينا والحكومه\nسألناها العدالة فاستبدت\nبسلطتها وعدتها جريمه\nفمن باللّه ينسب لاجترام\nأجنبي إن بقت للحق قيمه\nمَتى تُلقِ فَودَيها عَلى ظَهرِ ناهِضِ\n\nقَفا زيدٍ لقد جرّبت مني\nأنامل كالسياط ذ

In [5]:
# gather all unique characters within dataset
chars = sorted(list(set(poems)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
# target loss = -ln(1/vocab_size) = -ln(1/55) = 4.007

	
  ،؛؟ءآأؤإئابةتثجحخدذرزسشصضطظعغـفقكلمنهوىيًٌٍَُِّْچڑک
55


In [6]:
# tokenize chars by mapping to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # string --> list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # list of integers --> string

In [7]:
import torch
# encode dataset
data = torch.tensor(encode(poems), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([10000009]) torch.int64
tensor([ 2, 38, 41, 37, 13, 43,  2, 35, 37,  2, 37, 43,  2, 13, 43, 39,  2, 38,
        13,  2, 35, 21,  2, 36, 13, 39,  2, 38, 39,  2, 31, 40, 21,  2, 41, 17,
        43, 35,  2, 19, 13, 26, 13, 36,  2, 13, 39,  2, 16, 39, 25, 43,  2, 13,
        37, 22, 43,  2, 14, 43, 39, 43,  2, 41, 14, 43, 39, 36,  2, 38, 39,  2,
        19, 35, 41, 35,  2, 38, 13,  2, 38, 17, 37,  2, 41, 18, 40, 36,  2, 22,
        13,  2, 13, 37, 18, 38, 43,  2, 37,  2, 43, 36, 41, 39,  2, 38, 39,  2,
        13, 40, 37,  2, 13, 37, 31, 35, 41, 35,  2, 43, 14, 21, 41,  2, 34, 43,
        26, 23, 35,  2, 37, 31, 43, 41,  2, 39,  2, 28, 19, 43,  2, 41, 43, 26,
        23, 35, 39, 43,  2, 14, 23, 43, 35, 43,  2, 41, 24, 31, 38, 16,  2, 13,
        39, 36,  2, 24, 13,  7, 23, 43,  2, 34, 16, 23, 36, 16,  2, 31, 43, 39,
        43,  2, 37, 29, 23, 43, 35,  2, 41, 18, 31, 37, 16, 39, 43,  2, 13, 14,
        36, 43,  2, 31, 37, 43,  2, 36,  2, 38, 39,  2, 13, 37, 32, 23, 41, 14,
     

In [8]:
# train/val split = 90/10

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [22]:
# hparams

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
lr = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

cuda


In [10]:
# separate data into batches

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]) 
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y  = x.to(device), y.to(device)
    return x,y

In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [12]:
class Head(nn.Module):
    # single head of self-attention
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape

        k = self.key(x) # (B, T, C)
        q = self.query(x) # (B, T, C)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,C) @ (B,C,T) ---> (B,T,T) 
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v # (B,T,T) @ (B,T,C) ---> (B,T,C) 

        return out

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        
        return out

In [14]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        
        
    def forward(self, x):
        return self.net(x)

In [15]:
class Block(nn.Module):
#     Tranformer block: communication (self-attention) followed by computation (feed-forward)

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ff = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

In [16]:
class DiwanGPT(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        tok_emb = self.token_embedding(idx) # (B,T,C)
        pos_emb = self.pos_embedding(torch.arange(T, device=device)) # (T,C)
        
        # (B,T,C)
        x = tok_emb + pos_emb 
        x = self.blocks(x)
        x = self.ln_f(x)
        
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:] # (B,C)
            probs = F.softmax(logits, dim=-1)
#             sample from distribution using probs
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx

In [17]:
model = DiwanGPT()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters') # num of params in model

10.781239 M parameters


In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [24]:
# train
start_time = time.time()

for i in range(max_iters):
    if i % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        elapsed_time = time.time() - start_time
        print(f"Time elapsed: {str(datetime.timedelta(seconds=int(elapsed_time)))} step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Time elapsed: 0:00:32 step 0: train loss 1.7708, val loss 1.8093
Time elapsed: 0:03:26 step 500: train loss 1.7625, val loss 1.7974
Time elapsed: 0:06:20 step 1000: train loss 1.7416, val loss 1.7808
Time elapsed: 0:09:14 step 1500: train loss 1.7314, val loss 1.7738
Time elapsed: 0:12:08 step 2000: train loss 1.7186, val loss 1.7661
Time elapsed: 0:15:01 step 2500: train loss 1.7072, val loss 1.7536
Time elapsed: 0:17:55 step 3000: train loss 1.7014, val loss 1.7515
Time elapsed: 0:20:49 step 3500: train loss 1.6884, val loss 1.7418
Time elapsed: 0:23:43 step 4000: train loss 1.6821, val loss 1.7382
Time elapsed: 0:26:37 step 4500: train loss 1.6735, val loss 1.7332


In [26]:
# generate
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))
#open('/kaggle/working/generated_poetry.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

	ئِلُ
ذَلِلتُ لَهُ بِشراءَ بابِ وَمَعرِكَةٌ
تَكّو وُليتَها خُدودُ المُنى وَطَعا
وَهَل سَلَت بِهِ راقَت بِمُخَلَّدِهِ
مُنسِلُ السُحبِ البَدرِ مِن خَميرانِها
نَتَيمانِ إِلى إِبلَيهِ لَجاتِهِ
رَعَت بُغالِ الكَفاحِ الطَريفِ مُرتَصِبا
وَهَل عَيِّها بِالعَتيقِ باقِياً لِمَن تَرى
بِيَحيى نَعَمَ السَحاباءَ مالاقِيا
وَصَوتُ ذُرى صَوتِها وَاتَبَعَ بَعضَةً
عَرضاً كَأَنّي كانَ أَنّي أَغُمضَبا
لامَت في مُجاشِبِ النَكبَينِ بادِئاً
فَمالَمِصرُ نَسينُ الذِراعِ الضَعيفِ
هَذي حَلَمتِنا الرياحِ فَكَأَنَّما
حَوامِلَ غُربانا مَواطِنا السيفِ
إِن كانَتِ اللُجامُ لَها وَفَقَد رَآنا
عَبُسَ القَبيحَةَ غَمرُ الدُروبِ صِراحاً
رَجَعنا بِكُلِّ شَيبٍ أَم وَجيدٍ أَقومِ

إنّ سوادي المعذب حدّثنا
والذميل ينسانٌ أثواب
ولكن أهجز الشبابُ والشب
بانوا علي ثوب الغيب لأدم
في دولة الأطرار يا ملاذوني
ملك نار الكشاف بعدها وليد
قطعنا لهم ثم تاه لأنساب
إذا الذي كرمنا جنى دونكما
تطلع منها به المورود الجاري
فاطلب رحمة من تغاب لذيذ
أحنا مليكل العيد ذي أحباب
حتى يسخط العب رسما قبرا
ورؤيتها الاجفان ما زندنا
لكتافء عنا بعد نقع الشمائل
مت